In [1]:
import os, sys
sys.path.append("..")

# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [2]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [3]:
from importlib import reload
import paulipoly
reload(paulipoly)
from paulipoly import PauliPoly as NewPauliPoly
import ten_con
reload(ten_con)
from ten_con import _mat_to_coef_mat, _coef_to_mat

In [4]:
from scipy.sparse import coo_matrix, csr_matrix

In [5]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [6]:
n = 5
plist1 = [
    PauliElement(3, 5, n, 3),
    PauliElement(3, 1, n, 3),
    PauliElement(0, 4, n, 3),
    PauliElement(4, 2, n, 3),
]
plist2 = [
    PauliElement(2, 3, n, 3),
    PauliElement(2, 0, n, 3),
    PauliElement(0, 3, n, 3),
    PauliElement(3, 1, n, 3),
]
plist3 = [
    PauliElement(1, 1, n, 1),
    PauliElement(3, 0, n, 1),
    PauliElement(2, 2, n, 1),
    PauliElement(2, 5, n, 1),
    PauliElement(4, 4, n, 1),
    PauliElement(3, 5, n, 1),
    PauliElement(3, 1, n, 1),
]

In [7]:
qubits = 2
ndim = int(2**qubits)
A = np.random.rand(ndim,ndim)
B = np.random.rand(ndim,ndim)
A[A<0.4] = 0.
B[B<0.9] = 0.
test_ppoly = PauliPoly.from_coef_mat(A)

In [8]:
#plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix
cmat = csr_matrix(coef_mat)
100*cmat.data.size/(1024**2)

0.000762939453125

## Coef matrix to PauliPoly

In [9]:
%%timeit
cmat + cmat

18.6 μs ± 51 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%timeit
mat+mat

258 ns ± 2.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
%%timeit
PauliPoly.from_coef_mat(coef_mat) # Naive

125 μs ± 1.07 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit
NewPauliPoly(coef_mat) # scipy sparse routine.

2.81 μs ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Hermit matrix to PauliPoly

In [13]:
%%time
PauliPoly.from_matrix(mat)

CPU times: user 445 ms, sys: 8.32 ms, total: 453 ms
Wall time: 479 ms


PauliPoly(terms:12)[
((2.514195213000494-0.5012783154285163j), 'IZ'),
((-9.841316208e-314+4.9202767387e-314j), 'ZZ'),
((0.5012783154285163+2.514195213000494j), 'IX'),
((2.885131629128014+3.815730075531865j), 'IY'),
((4.9202761576e-314-5.4197e-319j), 'ZX'),
((4.2980049263e-314+4.2980049243e-314j), 'ZY'),
((-4.9202761576e-314+5.4197e-319j), 'XZ'),
((3.1565381181072274+1.9771689517718696j), 'YI'),
((-0.8758669697823427-1.0037000468825044j), 'YZ'),
((-9.841316208e-314+4.9202767387e-314j), 'XX'),
((-4.2980049243e-314+4.2980049263e-314j), 'XY'),
((1.0898910524694672+2.969458069134314j), 'YX')
]

In [14]:
%%timeit
NewPauliPoly.from_matrix(mat)

6.27 μs ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%%timeit
SparsePauliOp.from_operator(mat)

6.95 μs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Test

## PauliPoly to Hermit

In [16]:
# Naive
#pp001 = PauliPoly.from_coef_mat(A)
#pp002 = PauliPoly.from_coef_mat(B)
# Improved
npp001 = NewPauliPoly(A)
npp002 = NewPauliPoly(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)


In [17]:
%%timeit
npp001.matrix

2.48 μs ± 9.94 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [18]:
%%timeit
spa001.to_matrix()

15.4 μs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Addition

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [19]:
%%timeit
npp001+npp002

3.61 μs ± 6.56 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
%%timeit
spa001+spa002

8.53 μs ± 195 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Mat multiplication

In [21]:
#%%time
#pp001@pp002

In [22]:
from opttrot.pauli_utils import mat_decompose

In [27]:
mat = npp001.matrix@npp002.matrix
mat

matrix([[ 1.23638907+1.09440271j,  1.21890716+1.17418355j,
         -0.45495916-0.75333112j,  0.4240466 +0.57535726j],
        [-0.20537751-1.17418355j, -0.2917245 -1.09440271j,
         -0.4240466 +0.18919779j,  0.45495916-0.20177139j],
        [ 0.45495916-1.645464j  ,  0.4240466 -1.53252571j,
         -1.23638907+0.20741865j,  1.21890716-0.22253926j],
        [-0.4240466 -1.14636624j, -0.45495916-1.09390426j,
         -0.20537751+0.22253926j,  0.2917245 -0.20741865j]])

In [28]:
mat_decompose(mat)

matrix([[ 0.        +0.j        ,  1.48617002+1.36239932j,
         -2.73936826-0.95510251j, -0.49523473+1.7217235j ],
        [ 3.38945862+0.54088927j,  0.        +2.60364273j,
         -1.7217235 -1.41910217j, -1.46147806-1.46147806j],
        [ 2.73936826-0.95510251j,  1.7217235 +0.27708422j,
          1.88932913+0.j        , -1.39672281+1.39672281j],
        [ 2.19142112+1.7217235j , -0.35835858+0.35835858j,
          1.39672281+1.39672281j,  3.05622714+1.77396813j]])

In [72]:
mat.shape

(8, 8)

In [29]:
npp001@npp002

PauliPoly(terms:15)[
((1.4861700191891107+1.3623993234247673j), 'IZ'),
((-2.739368259433303-0.9551025067477077j), 'ZI'),
((-0.49523473287859854+1.721723499947384j), 'ZZ'),
((3.389458617396053+0.5408892747821747j), 'IX'),
(2.603642725235898j, 'IY'),
((-1.721723499947384-1.4191021692419628j), 'ZX'),
((-1.4614780553605702-1.4614780553605704j), 'ZY'),
((2.739368259433303-0.9551025067477077j), 'XI'),
((1.7217234999473838+0.27708421619925194j), 'XZ'),
((1.8893291316118757+0j), 'YI'),
((-1.3967228091876893+1.3967228091876893j), 'YZ'),
((2.1914211183198136+1.7217234999473843j), 'XX'),
((-0.35835858368115425+0.35835858368115403j), 'XY'),
((1.3967228091876893+1.3967228091876893j), 'YX'),
((3.056227135788077+1.7739681346035872j), 'YY')
]

In [62]:
from numpy import kron

In [63]:
l = np.matrix([[3, 3],[3, 3]], dtype=complex)
mat_decompose(kron(l, l))

matrix([[36. +0.j, 18.-18.j, 18.-18.j,  0.-18.j],
        [18.+18.j,  0. +0.j, 18. +0.j,  0. +0.j],
        [18.+18.j, 18. +0.j,  0. +0.j,  0. +0.j],
        [ 0.+18.j,  0. +0.j,  0. +0.j,  0. +0.j]])

In [30]:
%%timeit
npp001@npp002

14.4 μs ± 140 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
%%timeit
spa001@spa002

21.9 μs ± 1.03 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [49]:
sys.getsizeof(npp001@npp002)

48

In [50]:
sys.getsizeof(spa001@spa002)

48